<a href="https://colab.research.google.com/github/MLDreamer/Substack-AI-lessons/blob/main/RAG_Evaluation_The_Faithfulness_Recall_Frontier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
RAG Evaluation: The Faithfulness-Recall Frontier
==================================================

This notebook demonstrates why standard RAG metrics (ROUGE, BLEU, Answer F1) fail
and how Faithfulness and Context Recall expose the fundamental trade-off.

Based on real production experience building a pharmaceutical supply chain chatbot.


"""

# ==============================================================================
# SECTION 1: Environment Setup
# ==============================================================================

# Install required packages
# Uncomment and run if packages are missing:
# !pip install -q openai anthropic chromadb sentence-transformers pandas numpy matplotlib

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

# Vector store
import chromadb
from chromadb.utils import embedding_functions

print("✓ Environment setup complete")
print("\nConfigure your API key:")
print("os.environ['OPENAI_API_KEY'] = 'your-key-here'")
print("# or")
print("os.environ['ANTHROPIC_API_KEY'] = 'your-key-here'")

# ==============================================================================
# SECTION 2: Dataset - Real Examples from Production
# ==============================================================================

@dataclass
class QAExample:
    """
    Single question-answer example with context chunks and gold answer.
    Based on real pharmaceutical supply chain data.
    """
    question: str
    chunks: List[str]  # All available document chunks
    gold_answer: str
    category: str

# Real examples (anonymized) from the Moonylever deployment
PRODUCTION_EXAMPLES = [
    QAExample(
        question="Why was Route 47 flagged in the RCA report?",
        chunks=[
            "Route Root Cause Analysis procedures require evaluation of three primary factors: vehicle condition, driver compliance, and external disruptions.",
            "Driver overtime thresholds are tracked in the weekly logistics sheet. Column G records hours worked. Maximum allowable is 12 hours per shift.",
            "Fuel consumption variance is calculated as (actual - baseline) / baseline. Baseline values are established quarterly during efficiency reviews.",
            "Class-A restricted zones are defined in the regional compliance database. Routes intersecting these zones during peak hours require special authorization.",
            "Vehicle maintenance logs must be reviewed for any route flagged in RCA. Common issues include transmission problems, brake failures, and tire wear.",
            "Road closure notifications from municipal authorities are logged in the operations database. Unplanned closures require immediate route reassignment.",
            "Warehouse loading delays can cascade into route delays. Staffing shortages during peak periods are the most common cause of loading issues.",
            "Temperature monitoring deviations trigger automatic route flags. Cold chain integrity must be maintained for pharmaceutical cargo.",
        ],
        gold_answer="Route 47 was flagged due to vehicle breakdown (transmission failure), unexpected road closure from municipal construction, and delayed warehouse loading caused by staffing shortages.",
        category="RCA"
    ),

    QAExample(
        question="What are the driver scheduling constraints for route assignments?",
        chunks=[
            "Driver scheduling policy (revised March 2023): Maximum 5 consecutive working days. Mandatory 2 days rest after 5-day cycle.",
            "Previous policy (2021-2023): Maximum 6 consecutive working days. This policy was superseded due to fatigue-related incidents.",
            "Driver availability is checked in real-time via the fleet management system. Only drivers with valid certifications can be assigned pharmaceutical routes.",
            "Route assignments must account for driver experience level. New drivers (<6 months) cannot be assigned to high-value cargo routes.",
            "Emergency override procedures allow scheduling beyond normal constraints with approval from operations manager and documented justification.",
        ],
        gold_answer="Current policy allows maximum 5 consecutive working days with mandatory 2 days rest. Route assignments must verify driver certification and experience level. Emergency overrides require manager approval.",
        category="Scheduling"
    ),

    QAExample(
        question="What factors are considered in transport cost allocation?",
        chunks=[
            "Cost allocation methodology (Version 3.2, 2022): Direct costs include fuel, tolls, driver wages. Indirect costs include vehicle depreciation and insurance.",
            "Fuel costs are calculated using actual consumption data from vehicle telematics. Regional fuel price indices are updated weekly.",
            "Driver wage allocation uses hourly rates adjusted for overtime, night shifts, and hazard premiums for pharmaceutical cargo.",
            "Vehicle depreciation follows straight-line method over 8-year useful life. Insurance costs are prorated based on route risk classification.",
            "Toll expenses are captured via automated transponder systems. Manual toll submissions require photo documentation and approval.",
            "Historical cost allocation (2019-2021): Used estimated fuel consumption rather than actual data. Superseded by current telematics-based method.",
        ],
        gold_answer="Cost allocation includes direct costs (fuel, tolls, driver wages) and indirect costs (vehicle depreciation, insurance). Fuel uses actual telematics data. Driver wages account for overtime and hazard premiums. Current methodology effective 2022.",
        category="Cost"
    ),
]

print(f"✓ Loaded {len(PRODUCTION_EXAMPLES)} real production examples")
print(f"  Categories: {', '.join(set(ex.category for ex in PRODUCTION_EXAMPLES))}")

# ==============================================================================
# SECTION 3: Simple RAG System Implementation
# ==============================================================================

class SimpleRAG:
    """
    Minimal RAG system for demonstration.
    Uses Chroma for vector storage and sentence transformers for embeddings.
    """

    def __init__(self, embedding_model="all-MiniLM-L6-v2"):
        self.client = chromadb.Client()
        self.embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name=embedding_model
        )
        self.collection = None

    def index_documents(self, chunks: List[str], collection_name: str = "demo"):
        """Index document chunks into vector store"""
        try:
            self.client.delete_collection(name=collection_name)
        except:
            pass

        self.collection = self.client.create_collection(
            name=collection_name,
            embedding_function=self.embedding_fn
        )

        self.collection.add(
            documents=chunks,
            ids=[f"chunk_{i}" for i in range(len(chunks))]
        )

        return len(chunks)

    def retrieve(self, query: str, top_k: int = 5) -> List[str]:
        """Retrieve top-k most relevant chunks"""
        if self.collection is None:
            return []

        results = self.collection.query(
            query_texts=[query],
            n_results=min(top_k, self.collection.count())
        )

        return results['documents'][0] if results['documents'] else []

    def generate_answer(self, query: str, context: str, model="simulated") -> str:
        """
        Generate answer from context.

        In production, this calls your LLM (GPT-4, Claude, etc.)
        For this demo, we simulate generation based on context presence.
        """
        # For actual implementation, replace with:
        # response = openai.ChatCompletion.create(...)
        # return response.choices[0].message.content

        # Simulated generation for demo purposes
        return f"[Generated answer based on {len(context.split())} words of context]"

print("\n✓ RAG system class defined")
rag = SimpleRAG()
print("  Using embedding model: all-MiniLM-L6-v2")

# ==============================================================================
# SECTION 4: LLM Judge for Faithfulness and Context Recall
# ==============================================================================

class LLMJudge:
    """
    LLM-as-a-judge for extracting and verifying factual statements.

    In production, replace simulate_llm_call with actual API calls to GPT-4 or Claude.
    """

    @staticmethod
    def simulate_llm_call(prompt: str, task: str = "extract") -> str:
        """
        Simulate LLM responses for demo purposes.

        In production, replace with:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
        """
        # Simulated responses based on task type
        if task == "extract_answer":
            return """1. Route 47 was flagged due to vehicle breakdown
2. Transmission failure was the specific issue
3. Road closure occurred due to municipal construction
4. Warehouse loading delays were caused by staffing shortages"""

        elif task == "extract_gold":
            return """1. Route 47 flagging was due to vehicle breakdown
2. Breakdown was transmission failure
3. Road closure was from municipal construction
4. Delayed warehouse loading
5. Staffing shortage caused loading delay"""

        elif task == "verify":
            # Simulate verification based on simple heuristics
            return "YES" if "vehicle" in prompt.lower() or "route" in prompt.lower() else "NO"

        return ""

    def extract_statements(self, text: str, source: str = "answer") -> List[str]:
        """Extract atomic factual statements from text using LLM"""
        task = "extract_answer" if source == "answer" else "extract_gold"

        prompt = f"""Extract all factual claims from the following text as a numbered list.
Each claim should be atomic (one fact per line).
Do not add interpretation or inference.

Text: {text}

Provide only the numbered list of factual claims."""

        response = self.simulate_llm_call(prompt, task=task)

        # Parse numbered list
        statements = []
        for line in response.strip().split('\n'):
            line = line.strip()
            if line and (line[0].isdigit() or line.startswith('-')):
                # Remove numbering (e.g., "1. " or "- ")
                statement = line.split('.', 1)[-1].strip()
                if statement:
                    statements.append(statement)

        return statements

    def verify_statement(self, statement: str, context: str) -> bool:
        """Verify if statement is supported by context using LLM"""
        prompt = f"""Context: {context}

Statement: {statement}

Is this statement directly supported by the context above?
Consider:
- The statement must be explicitly present or clearly inferrable
- Paraphrasing is acceptable if meaning is preserved
- Do not accept speculation or unsupported claims

Answer only 'YES' or 'NO'."""

        response = self.simulate_llm_call(prompt, task="verify")
        return response.strip().upper() == "YES"

print("\n✓ LLM Judge class defined")
judge = LLMJudge()
print("  Note: Using simulated LLM for demo. Replace with actual API calls for production.")

# ==============================================================================
# SECTION 5: Faithfulness Calculation
# ==============================================================================

def calculate_faithfulness(answer: str, context: str, judge: LLMJudge,
                          verbose: bool = False) -> Tuple[float, Dict]:
    """
    Calculate Faithfulness score: F = (supported statements) / (total statements)

    Faithfulness measures: "Did the model make anything up?"
    - F = 1.0: Every claim is traceable to context (no hallucinations)
    - F = 0.6: 40% of the answer is fabricated

    Args:
        answer: Generated answer from RAG system
        context: Retrieved context (concatenated chunks)
        judge: LLM judge instance
        verbose: Print detailed verification results

    Returns:
        (faithfulness_score, details_dict)
    """
    # Extract statements from answer
    statements = judge.extract_statements(answer, source="answer")

    if not statements:
        return 1.0, {"statements": 0, "supported": 0, "unsupported_statements": []}

    # Verify each statement
    supported = []
    unsupported = []

    for stmt in statements:
        is_supported = judge.verify_statement(stmt, context)

        if is_supported:
            supported.append(stmt)
        else:
            unsupported.append(stmt)

        if verbose:
            status = "✓" if is_supported else "✗"
            print(f"  {status} {stmt[:80]}...")

    faithfulness = len(supported) / len(statements)

    details = {
        "statements": len(statements),
        "supported": len(supported),
        "unsupported": len(unsupported),
        "unsupported_statements": unsupported
    }

    return faithfulness, details

print("\n✓ Faithfulness calculation function defined")
print("  Formula: F = (statements supported by context) / (total statements in answer)")

# ==============================================================================
# SECTION 6: Context Recall Calculation
# ==============================================================================

def calculate_context_recall(gold_answer: str, context: str, judge: LLMJudge,
                             verbose: bool = False) -> Tuple[float, Dict]:
    """
    Calculate Context Recall score: R = (gold statements in context) / (total gold statements)

    Context Recall measures: "Did retrieval pull all necessary information?"
    - R = 1.0: Context contains everything needed for complete answer
    - R = 0.5: Retriever only found half the required information

    Args:
        gold_answer: Reference/ground truth answer
        context: Retrieved context (concatenated chunks)
        judge: LLM judge instance
        verbose: Print detailed attribution results

    Returns:
        (recall_score, details_dict)
    """
    # Extract statements from gold answer
    gold_statements = judge.extract_statements(gold_answer, source="gold")

    if not gold_statements:
        return 1.0, {"gold_statements": 0, "found": 0, "missing_statements": []}

    # Check which statements are present in context
    found = []
    missing = []

    for stmt in gold_statements:
        is_present = judge.verify_statement(stmt, context)

        if is_present:
            found.append(stmt)
        else:
            missing.append(stmt)

        if verbose:
            status = "✓" if is_present else "✗"
            print(f"  {status} {stmt[:80]}...")

    recall = len(found) / len(gold_statements)

    details = {
        "gold_statements": len(gold_statements),
        "found": len(found),
        "missing": len(missing),
        "missing_statements": missing
    }

    return recall, details

print("\n✓ Context Recall calculation function defined")
print("  Formula: R = (gold statements found in context) / (total gold statements)")

# ==============================================================================
# SECTION 7: The Problem - Standard Metrics Miss Hallucinations
# ==============================================================================

print("\n" + "="*80)
print("DEMONSTRATION 1: Why Standard Metrics Fail")
print("="*80)

# Simulate the Route 47 example
example = PRODUCTION_EXAMPLES[0]

# Low K (K=1): Safe but shallow
print("\n--- Configuration: K=1 (retrieve 1 chunk) ---")
rag.index_documents(example.chunks, "demo_k1")
retrieved_k1 = rag.retrieve(example.question, top_k=1)
context_k1 = "\n\n".join(retrieved_k1)

print(f"Retrieved {len(retrieved_k1)} chunk(s)")
print(f"Context preview: {context_k1[:150]}...")

# Simulate answer with high grounding, low completeness
simulated_answer_k1 = "Route RCA requires evaluation of vehicle condition, driver compliance, and external disruptions per standard procedures."

print(f"\nGenerated answer: {simulated_answer_k1}")

f_k1, f_details_k1 = calculate_faithfulness(simulated_answer_k1, context_k1, judge, verbose=True)
r_k1, r_details_k1 = calculate_context_recall(example.gold_answer, context_k1, judge, verbose=True)

print(f"\nFaithfulness (F): {f_k1:.2f} ← High grounding")
print(f"Context Recall (R): {r_k1:.2f} ← Low completeness")
print(f"User Experience: Correct but useless. Missing critical specifics.")

# High K (K=7): Complete but risky
print("\n--- Configuration: K=7 (retrieve 7 chunks) ---")
rag.index_documents(example.chunks, "demo_k7")
retrieved_k7 = rag.retrieve(example.question, top_k=7)
context_k7 = "\n\n".join(retrieved_k7)

print(f"Retrieved {len(retrieved_k7)} chunk(s)")

# Simulate answer with fabrication due to noise
simulated_answer_k7 = "Route 47 was flagged due to driver overtime exceeding 12 hours (column G), fuel variance 18% above Q2 baseline, and intersection with Class-A restricted zones. Additionally, vehicle transmission issues and road closures contributed."

print(f"\nGenerated answer: {simulated_answer_k7}")

f_k7, f_details_k7 = calculate_faithfulness(simulated_answer_k7, context_k7, judge, verbose=False)
r_k7, r_details_k7 = calculate_context_recall(example.gold_answer, context_k7, judge, verbose=False)

print(f"\nFaithfulness (F): {f_k7:.2f} ← Dropped (hallucinations)")
print(f"Context Recall (R): {r_k7:.2f} ← High completeness")
print(f"User Experience: Comprehensive but contains fabricated details.")

print(f"\n⚠️  The Problem: More context increased completeness but decreased grounding.")

# ==============================================================================
# SECTION 8: Mapping the Complete Frontier
# ==============================================================================

print("\n" + "="*80)
print("DEMONSTRATION 2: The Faithfulness-Recall Frontier")
print("="*80)

def evaluate_configuration(example: QAExample, k: int, rag: SimpleRAG,
                          judge: LLMJudge) -> Dict:
    """Evaluate RAG system at a specific K value"""

    # Index and retrieve
    rag.index_documents(example.chunks, f"demo_k{k}")
    retrieved = rag.retrieve(example.question, top_k=k)
    context = "\n\n".join(retrieved)

    # Simulate generation with varying quality based on K
    # In production, this would be actual LLM generation
    if k <= 2:
        # Low K: high faithfulness, low recall
        answer = "Basic procedural answer covering minimal facts."
        f_sim, r_sim = 0.95 - (k * 0.02), 0.45 + (k * 0.08)
    elif k <= 4:
        # Medium K: balanced
        answer = "More comprehensive answer with multiple factors."
        f_sim, r_sim = 0.93 - (k * 0.04), 0.55 + (k * 0.06)
    else:
        # High K: low faithfulness, high recall
        answer = "Very detailed answer incorporating many sources."
        f_sim, r_sim = 0.95 - (k * 0.05), 0.60 + (k * 0.05)

    # Calculate metrics
    faithfulness, f_details = calculate_faithfulness(answer, context, judge)
    recall, r_details = calculate_context_recall(example.gold_answer, context, judge)

    # Use simulated values for consistent demo
    faithfulness, recall = f_sim, r_sim

    return {
        'k': k,
        'faithfulness': faithfulness,
        'recall': recall,
        'ratio': faithfulness / recall if recall > 0 else float('inf'),
        'chunks_retrieved': len(retrieved)
    }

# Evaluate across K range
k_range = [1, 2, 3, 4, 5, 6, 7, 8]
results = []

print("\nEvaluating across K ∈ [1, 2, 3, 4, 5, 6, 7, 8]")
print("-" * 60)
print(f"{'K':<4} {'F':<6} {'R':<6} {'F/R':<6} {'Status'}")
print("-" * 60)

F_MIN = 0.90  # Minimum acceptable faithfulness

for k in k_range:
    result = evaluate_configuration(PRODUCTION_EXAMPLES[0], k, rag, judge)
    results.append(result)

    status = "✓" if result['faithfulness'] >= F_MIN else "✗"
    print(f"{result['k']:<4} {result['faithfulness']:.3f}  {result['recall']:.3f}  "
          f"{result['ratio']:.3f}  {status}")

# Find optimal configuration
valid_configs = [r for r in results if r['faithfulness'] >= F_MIN]

if valid_configs:
    optimal = max(valid_configs, key=lambda x: x['recall'])
    print(f"\n🎯 OPTIMAL: K={optimal['k']}")
    print(f"   Faithfulness: {optimal['faithfulness']:.3f} (meets {F_MIN} threshold)")
    print(f"   Recall: {optimal['recall']:.3f} (maximized)")
    print(f"   F/R Ratio: {optimal['ratio']:.3f}")
else:
    print("\n⚠️  No configuration meets faithfulness threshold")

# ==============================================================================
# SECTION 9: Visualization
# ==============================================================================

print("\n" + "="*80)
print("VISUALIZATION: The F-R Trade-off")
print("="*80)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Faithfulness vs Recall frontier
f_vals = [r['faithfulness'] for r in results]
r_vals = [r['recall'] for r in results]
k_vals = [r['k'] for r in results]

# Scatter plot with color gradient
scatter = ax1.scatter(r_vals, f_vals, c=k_vals, cmap='viridis',
                     s=150, alpha=0.7, edgecolors='black', linewidth=1.5)
ax1.plot(r_vals, f_vals, 'k--', alpha=0.3, linewidth=1)

# Annotate points
for i, k in enumerate(k_vals):
    ax1.annotate(f'K={k}', (r_vals[i], f_vals[i]),
                xytext=(8, 0), textcoords='offset points',
                fontsize=9, fontweight='bold')

# Mark optimal point
if valid_configs:
    optimal_r = optimal['recall']
    optimal_f = optimal['faithfulness']
    ax1.scatter([optimal_r], [optimal_f], s=400, marker='*',
               c='red', edgecolors='black', linewidth=2.5,
               label=f'Optimal (K={optimal["k"]})', zorder=10)

# Threshold line
ax1.axhline(y=F_MIN, color='red', linestyle='--', linewidth=2,
           alpha=0.6, label=f'F_min = {F_MIN}')

# Styling
ax1.set_xlabel('Context Recall (R)', fontsize=13, fontweight='bold')
ax1.set_ylabel('Faithfulness (F)', fontsize=13, fontweight='bold')
ax1.set_title('The Optimal Frontier: Grounding vs Completeness',
             fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, linestyle=':')
ax1.legend(fontsize=10, loc='lower right')
ax1.set_xlim(0.35, 1.0)
ax1.set_ylim(0.55, 1.0)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax1)
cbar.set_label('Top-K', fontsize=11, fontweight='bold')

# Plot 2: F/R Ratio across K
ratios = [r['ratio'] for r in results]

ax2.plot(k_vals, ratios, 'o-', linewidth=2.5, markersize=10,
        color='steelblue', markeredgecolor='black', markeredgewidth=1.5)

if valid_configs:
    ax2.scatter([optimal['k']], [optimal['ratio']], s=400, marker='*',
               c='red', edgecolors='black', linewidth=2.5, zorder=10,
               label='Optimal')

# Styling
ax2.set_xlabel('Top-K', fontsize=13, fontweight='bold')
ax2.set_ylabel('F/R Ratio (lower = better)', fontsize=13, fontweight='bold')
ax2.set_title('Efficiency: Grounding per Unit Completeness',
             fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, linestyle=':')
ax2.set_xticks(k_vals)
ax2.legend(fontsize=10)

plt.tight_layout()
plt.savefig('rag_faithfulness_recall_frontier.png', dpi=300, bbox_inches='tight')
print("\n✓ Visualization saved: rag_faithfulness_recall_frontier.png")
plt.show()

# ==============================================================================
# SECTION 10: Production Implementation Guide
# ==============================================================================

print("\n" + "="*80)
print("PRODUCTION IMPLEMENTATION")
print("="*80)

production_code = '''
# Step 1: Replace simulated LLM with actual API calls

import openai  # or import anthropic

def extract_statements_production(text: str, api_key: str) -> List[str]:
    """Extract factual statements using GPT-4"""

    client = openai.OpenAI(api_key=api_key)

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{
            "role": "user",
            "content": f"""Extract all factual claims from this text as a numbered list.
Each claim should be atomic (one fact per line).

Text: {text}

Provide only the numbered list."""
        }],
        temperature=0
    )

    # Parse response
    statements = []
    for line in response.choices[0].message.content.split('\\n'):
        line = line.strip()
        if line and line[0].isdigit():
            statement = line.split('.', 1)[-1].strip()
            if statement:
                statements.append(statement)

    return statements


def verify_statement_production(statement: str, context: str, api_key: str) -> bool:
    """Verify statement support using GPT-4"""

    client = openai.OpenAI(api_key=api_key)

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{
            "role": "user",
            "content": f"""Context: {context}

Statement: {statement}

Is this statement directly supported by the context?
Answer only YES or NO."""
        }],
        temperature=0
    )

    return response.choices[0].message.content.strip().upper() == "YES"


# Step 2: Evaluate your RAG system

def evaluate_rag_production(questions: List[str],
                           gold_answers: List[str],
                           your_rag_system,
                           k_values: List[int],
                           api_key: str):
    """Complete evaluation across K configurations"""

    results = []

    for k in k_values:
        f_scores = []
        r_scores = []

        for q, gold in zip(questions, gold_answers):
            # Your RAG system
            context = your_rag_system.retrieve(q, top_k=k)
            answer = your_rag_system.generate(q, context)

            # Calculate metrics
            answer_stmts = extract_statements_production(answer, api_key)
            gold_stmts = extract_statements_production(gold, api_key)

            # Faithfulness
            supported = sum(1 for s in answer_stmts
                          if verify_statement_production(s, context, api_key))
            f = supported / len(answer_stmts) if answer_stmts else 1.0

            # Context Recall
            found = sum(1 for s in gold_stmts
                       if verify_statement_production(s, context, api_key))
            r = found / len(gold_stmts) if gold_stmts else 1.0

            f_scores.append(f)
            r_scores.append(r)

        results.append({
            'k': k,
            'faithfulness': np.mean(f_scores),
            'recall': np.mean(r_scores)
        })

    return results


# Step 3: Find optimal configuration

def find_optimal_k(results: List[Dict], f_min: float = 0.90) -> int:
    """Find K that maximizes recall while maintaining faithfulness >= f_min"""

    valid = [r for r in results if r['faithfulness'] >= f_min]

    if not valid:
        print(f"Warning: No configuration meets f_min={f_min}")
        return results[0]['k']

    optimal = max(valid, key=lambda x: x['recall'])
    return optimal['k']
'''

print(production_code)

# ==============================================================================
# SECTION 11: Key Takeaways
# ==============================================================================

print("\n" + "="*80)
print("KEY TAKEAWAYS")
print("="*80)

print("""
1. Standard Metrics Are Misleading
   - ROUGE, BLEU, and Answer F1 measure textual similarity, not truth
   - High scores can coexist with severe hallucinations
   - These metrics cannot isolate retrieval vs generation failures

2. Two Dimensions Matter
   - Faithfulness (F): Proportion of answer statements supported by context
   - Context Recall (R): Proportion of gold answer present in context
   - These are orthogonal: you can fail on one while succeeding on the other

3. The Trade-off Is Fundamental
   - Increasing Top-K increases recall but decreases faithfulness
   - The relationship is non-linear and monotonic
   - You cannot maximize both simultaneously with current architectures

4. The Strategy Is Mathematical
   - Define minimum acceptable faithfulness (F_min) based on use case
   - Maximize recall subject to F >= F_min constraint
   - Track F/R ratio to measure grounding efficiency

5. Production Requirements
   - Replace simulated LLM with actual API calls (GPT-4, Claude)
   - Evaluate on domain-specific data with expert-validated gold answers
   - Monitor both metrics in production, not just end-to-end accuracy
   - Set alerts for faithfulness drops below threshold

6. The Next Frontier
   - Current RAG architectures force the F-R trade-off
   - Future systems must break this: increase R without sacrificing F
   - Possible directions: graph RAG, multi-hop retrieval, agentic reasoning
   - But these introduce new failure modes that need measurement

7. What Changed For Me
   - Before: Trusted ROUGE scores, deployed broken system
   - After: Measure F and R, understand true performance
   - The shift: From optimizing similarity to optimizing truth
""")

print("\n" + "="*80)
print("For the complete mathematical treatment and real production narrative:")
print("[Link to Substack Article]")
print("="*80)